In [63]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import json
import copy
import re
import csv
from keras.utils import to_categorical
import codecs
import keras.preprocessing.text as kpt
from keras.preprocessing.sequence import pad_sequences
import pickle
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation , LSTM , Input , Embedding
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, concatenate, Activation, Average
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout , Bidirectional
from keras.layers import Flatten , LSTM , Reshape, LeakyReLU
from keras.layers.embeddings import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.regularizers import L1L2
from keras import optimizers
from keras.callbacks import CSVLogger
import json
import numpy as np
import keras
import keras.preprocessing.text as kpt
#from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json

In [64]:
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [65]:
#train.tsv
train=pd.read_csv('train_en.tsv',delimiter='\t',encoding='utf-8')
print(list(train.columns.values)) #file header

[u'id', u'text', u'HS', u'TR', u'AG']


In [66]:
train.count()

id      10000
text    10000
HS      10000
TR      10000
AG      10000
dtype: int64

In [67]:
my_df = train

In [68]:
print(my_df.text.head(2))

0    Hurray, saving us $$$ in so many ways @potus @...
1    ;Why would young fighting age men be the vast ...
Name: text, dtype: object


In [69]:
my_word_stop = ['the','in','of','is','a','to','an','be','are','for','was','it','as','on', 'so', 'at']
# import re

In [70]:
# def tokenize_data(data):
#     tokenized_data = []
#     for line in data:
# #         for i in line.split():
# #             if i[:1] == '#':
# #                 line = line.strip() + ' ' + i[1:]
# #             elif i[:1] == '@':
# #                 line = line.strip() + ' ' + i[1:]
#         line = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*","",line)
#         line = re.sub(r"(.)\1+", r"\1",line)
#         #line = re.sub(r"#\S+", "",line)
#         #line = re.sub(r"@\S+", "",line)
#         #line = re.sub(r'([!.,?();*:\[\]":\”\“])([\w!.,?();*\[\]":\”\“])', r'\1 \2',line)
#         #line = " ".join([word for word in line.split() if word not in my_word_stop])
#         tokenized_data.append(line)
#     return tokenized_data

In [71]:
# ls = []
# Xs = []
# ls = tokenize_data(my_df.text)

In [72]:
# print(ls[0])
# print(my_df.text[0])

In [73]:
with open('segmentation_train_dev.csv', 'w') as f:
    writer = csv.writer(f)
    for row in my_df.text:
        row = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*","url",row)
        row = " ".join([word for word in row.split() if word not in my_word_stop])
        row = re.sub(r"(.)\1+", r"\1",row)
        #" ".join([word for word in line.split()
        rest_array = [text.encode("utf8") for text in row]
        rest_array = "".join(rest_array)
        #rest_array = rest_array.replace(',','')
        writer.writerow(rest_array)
        #print(rest_array)
f.close()

In [74]:
# print(row)
# print(rest_array[2])

In [75]:
train=pd.read_csv('segmentation_train_dev.csv',delimiter='\t',header=None)

In [76]:
train.count()

0    10000
dtype: int64

In [77]:
from keras.preprocessing.text import Tokenizer

In [78]:
train[0] = train[0].astype('str')

In [79]:
train_x = train[0]

In [80]:
train_y = my_df.AG

In [81]:
my_df.HS.count()

10000

In [82]:
# create a new Tokenizer
tokenizer = Tokenizer(lower=False,filters=',',char_level=False)
# feed our posts to the Tokenizer
tokenizer.fit_on_texts(train_x)

In [83]:
#Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index
# Let's save this out so we can use it later
with open('dictionary.p', 'wb') as fp:
    pickle.dump(dictionary, fp)

# with open('dictionary.json', 'w') as dictionary_file:
#     json.dump(dictionary, dictionary_file, ensure_ascii=False)

def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    temp_wordIndices = []
    for word in kpt.text_to_word_sequence(text,filters=',',lower=False):
        if word in dictionary:
            temp_wordIndices.append(dictionary[word])
    return temp_wordIndices

allWordIndices = []
# for each post, change each token to its ID in the Tokenizer's word_index
for text in train_x:
    #print(text)
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

# now we have a list of all posts converted to index arrays.
# cast as an array for future usage.
allWordIndices = np.asarray(allWordIndices)


In [84]:
print('Found %s unique tokens.' % len(dictionary))

Found 169 unique tokens.


In [85]:
# create one-hot matrices out of the indexed posts
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
# treat the labels as categories
train_y = to_categorical(train_y, 2)

In [86]:
train_x.shape[1]
#train_y.shape

170

In [87]:
from keras.layers.advanced_activations import LeakyReLU

In [88]:
model = Sequential()

model.add(Dense(100, activation='relu',input_shape=(train_x.shape[1],)))
#model.add(Conv1D(250, 3, padding='valid', activation='relu', strides=1,input_shape=(train_x.shape[1],)))
#model.add(GlobalMaxPooling1D())
#model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.2))
model.add(Dense(200, activation='relu'))
#model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.3))

model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

print(model.summary())

filepath="sequencing_the_data_try_n_error.{epoch:02d}-{val_loss:.4f}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger('final_log.csv', append=True, separator=';')

model.fit(train_x, train_y,
    batch_size=1,
    epochs=2,
    verbose=1,
    validation_split=0.20,
    shuffle=True,callbacks = [csv_logger])

model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')

print('saved model!')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 100)               17100     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 200)               20200     
_________________________________________________________________
dropout_6 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 402       
Total params: 37,702
Trainable params: 37,702
Non-trainable params: 0
_________________________________________________________________
None
Train on 8000 samples, validate on 2000 samples
Epoch 1/2
8000/8000 [==============================] - 9s 1ms/step - loss: 0.4507 - a

In [89]:
#loading

In [90]:
#test.tsv
test=pd.read_csv('test_en.tsv',delimiter='\t',encoding='utf-8')
print(list(test.columns.values)) #file header

[u'id', u'text']


In [91]:
with open('segmentation_test.csv', 'w') as f:
    writer = csv.writer(f)
    for row in test.text:
        row = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*","url",row)
        row = " ".join([word for word in row.split() if word not in my_word_stop])
        row = re.sub(r"(.)\1+", r"\1",row)
        #" ".join([word for word in line.split()
        rest_array = [text.encode("utf8") for text in row]
        rest_array = "".join(rest_array)
        writer.writerow(rest_array)
f.close()

In [92]:
test_final=pd.read_csv('segmentation_test.csv',delimiter='\t',header=None)
print(list(test_final.columns.values)) #file header
test_final[0] = test_final[0].astype('str')

[0]


In [93]:
# we're still going to use a Tokenizer here, but we don't need to fit it
tokenizer = Tokenizer(num_words=train_x.shape[1])
# for human-friendly printing
#labels = ['OAG','CAG','NAG']

# read in our saved dictionary

with open('dictionary.p', 'rb') as fp:
    dictionary = pickle.load(fp)
    
    
# with open('dictionary.json', 'r') as dictionary_file:
#     dictionary = json.load(dictionary_file)

# this utility makes sure that all the words in your input
# are registered in the dictionary
# before trying to turn them into a matrix.
not_found_word_list = []
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text,filters='',lower=False,split=',')
    wordIndices = []
    no_word = 0
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
        else:
            #print("'%s' not in training corpus; ignoring." %(word))
            if word == "":
                not_found_word_list.append(word)
                no_word = no_word + 1
    
    return wordIndices,no_word

# read in your saved model structure
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
# and create a model from that
model = model_from_json(loaded_model_json)
# and weight your nodes with your saved values
model.load_weights('model.h5')
with open('fileName.csv', 'w') as f:
    count=0
    no_words = 0
    for row in test_final[0]:
        # okay here's the interactive part
        evalSentence = row
        # format your input for the neural net
        testArr,no_word = convert_text_to_index_array(evalSentence)
        input = tokenizer.sequences_to_matrix([testArr], mode='binary')
        # predict which bucket your input belongs in
        pred = model.predict(input)
        # and print it for the humons
        f.write(str(np.argmax(pred)) + "\n")
        #f.write(np.argmax(pred).astype('str') + "\n")
        #f.write(pred + "\n")
        count+=1
        no_words+=no_word
f.close()
print(count)
print("word not found : ", no_words)
with open('not_found_word_list.csv', 'w') as f:
    for word in not_found_word_list:
        f.write(str(word)+"\n")
f.close()

3000
('word not found : ', 0)
